# Ant colony optimization y el problema del viajante.


**Ariel Rossanigo**

### Objetivos de la charla

* Contar que es Ant Colony Optimization
* Mostrar como funciona el algoritmo resolviendo el problema del viajante

## Ant colony optimization

* Marco Dorigo - 1992
* Algoritmo de optimización inspirado en el comportamiento de las hormigas 
* Utiliza la noción de depositar feromona donde la hormiga va pasando
* Problemas que pueden ser reducidos a buscar caminos en un grafo

## La idea básica:

    Distribuir aleatoriamente hormigas en el terreno
    Mientras el comportamiento no se estabilice:
    
        Calcular soluciones
        Realizar cambios en el ambiente
        Actualizar los depósitos de feromona.
  


## Travelling salesman problem (TSP)

* Hay N ciudades a visitar conectadas entre si
* La solución es el orden en que hay que visitar las ciudades de manera tal de pasar una única vez por cada una 
* Una solución óptima es la que demanda recorrer menor distancia 


### Nuestro mapa de ejemplo

In [1]:
import json
from itertools import combinations
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
 
CITIES = json.load(open('cities.json'))

p = figure(title="Current map")

Xs = [c['x'] for c in CITIES]
Ys = [c['y'] for c in CITIES]
names = [c['name'] for c in CITIES]

p= figure()
p.circle(Xs, Ys)
p.text(Xs, Ys, text=names, text_font_size="10pt")

for c1, c2 in combinations(CITIES, 2):
    x1, y1 = c1['x'], c1['y']
    x2, y2 = c2['x'], c2['y']
    p.segment(x1, y1, x2, y2, line_width=0.5)

Loading BokehJS ...

In [2]:
show(p)

### Implementando ACO para TSP.

In [3]:
import math
import random
from functools import lru_cache

CITIES_BY_ID = {x['id']:x for x in CITIES}

@lru_cache()
def distance(id1, id2):
    c1, c2 = CITIES_BY_ID[id1], CITIES_BY_ID[id2]
    return math.sqrt((c1['x'] - c2['x'])**2 + (c1['y'] - c2['y'])**2)


In [4]:
class Ant:
    def __init__(self, cities):
        self.last_tour = []
        self.last_cost = 0
        self.cities = cities
    
    def make_tour(self, pheromone, initial_city=None):
        not_visited = self.cities[:]
        if initial_city is None:
            initial_city = random.choice(not_visited)

        current_city = initial_city
        not_visited.remove(current_city)
        self.last_tour = [current_city]
        
        dead = False
        while not_visited and not dead:
            next_city = choose_with_probabilities(current_city, not_visited, pheromone)
            if next_city:
                self.last_tour.append(next_city)
                not_visited.remove(next_city)        
                current_city = next_city
            else:
                dead = True
                self.last_tour = []
        if not dead:
            self.last_tour.append(initial_city)
            self.last_cost = path_cost(self.last_tour)

In [5]:
def compute_probabilities(city, not_visited, pheromone, alpha=1, beta=1):
    N = [x for x in not_visited if x!= city]
    total = sum([(pheromone[(city, n)] ** alpha) * 
                 ((1/distance(city, n)) ** beta)
                 for n in N
        ])
    return [(n, (pheromone[(city, n)] ** alpha * 
                 (1/distance(city, n)) ** beta)/total) 
        for n in N
    ]

def choose_with_probabilities(current_city, not_visited, pheromone):
    r = random.random()
    accum_prob = 0
    for city, prob in compute_probabilities(current_city, not_visited, pheromone):
        accum_prob += prob
        if r <= accum_prob:
            return city
    return None
        
def path_cost(path):
    return sum(distance(c1, c2) for c1, c2 in zip(path[:-1], path[1:]))

In [6]:
class PheromoneDict(dict):
    def __init__(self, cities, initial_pheromone):
        super().__init__()
        for c1, c2 in combinations(cities, 2):
            self[(c1, c2)] = initial_pheromone
        
    def __setitem__(self, k, v):
        k = tuple(sorted(k))
        super().__setitem__(k, v)
        
    def __getitem__(self, k):
        k = tuple(sorted(k))
        return super().__getitem__(k)

In [9]:
# initialize parameters
EVAPORATION_COEF = 0.8
cities = [x['id'] for x in CITIES[:]]
current_pheromones = PheromoneDict(cities, 1)
ants = [Ant(cities) for x in range(20)]

iterations = []
avg_costs = []

In [10]:
iteration = 0
stagnation = False
while (iteration < 200 and not stagnation):
    # every ant makes his solution
    for a in ants:
        a.make_tour(current_pheromones)        
    
    delta_pheromones = PheromoneDict(cities, 0)
    total_cost = 0.0
    ants_no_dead = [x for x in ants if len(x.last_tour) > 0]
    selected_ants = sorted(ants_no_dead, key=lambda x: x.last_cost)[:10]
    for a in selected_ants:
        L = a.last_cost
        total_cost += L
        for c1, c2 in zip(a.last_tour[:-1], a.last_tour[1:]):
            delta_pheromones[(c1, c2)] += 1/L        
    
    for k, delta in delta_pheromones.items():
        current_pheromones[k] = (EVAPORATION_COEF * current_pheromones[k] + 
                                 delta / len(selected_ants))
        
    avg_costs.append(total_cost / len(selected_ants))
    stagnation = len(avg_costs) > 20 and all(avg_costs[-1] == x for x in avg_costs[-10:])
    iteration += 1

In [12]:
p = figure(title="AVG cost by iteration")

p.line(range(len(avg_costs)), avg_costs)
show(p)

In [20]:
p = figure(title="Current map")

Xs = [c['x'] for c in CITIES]
Ys = [c['y'] for c in CITIES]
names = [c['name'] for c in CITIES]

p= figure()
p.circle(Xs, Ys)
p.text(Xs, Ys, text=names, text_font_size="10pt")

for (id1, id2), pheromone in current_pheromones.items():
    c1, c2 = CITIES_BY_ID[id1], CITIES_BY_ID[id2]
    x1, y1 = c1['x'], c1['y']
    x2, y2 = c2['x'], c2['y']
    if pheromone > 0.00001:
        p.segment(x1, y1, x2, y2, line_width=10000*pheromone)

In [21]:
show(p)

### Gracias! ¿Preguntas?

Si me quieren contactar:

* arielrossanigo@gmail.com
* @arielrossanigo
* https://github.com/arielrossanigo